In [21]:
from rdflib import Graph, URIRef, RDF

KG_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test2_filled.ttl"
g = Graph()
g.parse(KG_PATH, format="turtle")

Q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ap:  <http://www.semanticweb.org/AgentProgramParams/>

SELECT ?pou ?name ?lang ?code
WHERE {
  ?pou rdf:type ?t .
  FILTER(?t IN (ap:class_Program, ap:class_FBType)) .
  ?pou ap:dp_hasPOUName ?name .
  ?pou ap:dp_hasPOUCode ?code .
  FILTER(CONTAINS(STR(?code), "METHOD")) .
  OPTIONAL { ?pou ap:dp_hasPOULanguage ?lang . }
}
"""
rows = []
for r in g.query(Q):
    rows.append({
        "pou_iri": str(r.pou),
        "name": str(r.name),
        "lang": str(r.lang) if r.lang else None,
        "code": str(r.code) if r.code else None,
    })
    print(str(r.pou), str(r.name), str(r.lang) if r.lang else None, str(r.code) if r.code else None)

print("POUs:", len(rows))
print([x["name"] for x in rows[:5]])


http://www.semanticweb.org/AgentProgramParams/FBType_FB_MyOpcUaMethod FB_MyOpcUaMethod ST // METHOD Multiply of FB_MyOpcUaMethod
Multiply := a * b;
http://www.semanticweb.org/AgentProgramParams/FBType_JobMethode_Schablone JobMethode_Schablone ST // POU JobMethode_Schablone body
JobMethode_Name(
    MethodCall := MethodCall_Name,
    NotAus_Signal := NotAusSignal,
    Direction_01 := Direction_01,
    Direction_02 := Direction_02,
    LightSensor_01 := LightSensor_01,
    LightSensor_02 := LightSensor_02,
    TS_Endschalter_01 := TS_Endschalter_01,
    AutomaticMode := _REFACTOR_,
    TS_Endschalter_02 := TS_Endschalter_02,
    TS_EndschalterEntlastung := TS_EndschalterEntlastung,
    TOF_Time_Motoransteuerung := TOF_Time_Motoransteuerung,
    TOF_Time_EndschalterEntlastung := TOF_Time_EndschalterEntlastung
);
Motor_Ansteuerung01 := JobMethode_Name.Motor_Direction01;

// METHOD Abort of JobMethode_Schablone
callCounterAbort := callCounterAbort + 1;

// METHOD CheckState of JobMethode_Sc

In [14]:
from groq import Groq
import json
import time

api_key_path = r"C:\Users\Alexander Verkhov\Desktop\APIKey_Groq.txt"
with open(api_key_path, "r", encoding="utf-8") as f:
    groq_api_key = f.read().strip()

client = Groq(api_key=groq_api_key)

CSS_DEFINITION = """
CSS-Modell (kurz):
- Capability: implementierungsunabhängige Beschreibung einer Funktion (was soll möglich sein).
- Skill: implementierte Funktion, die eine Capability konkret ausführt (wie wird es ausgeführt).
- Service: angebotene Schnittstelle, über die ein Skill genutzt wird (z.B. API, OPC UA, PLC-Job-Interface).
Bewerte anhand des Codes, ob dieser POU einen ausführbaren Skill (Job) darstellt.
"""

In [15]:
SKILL_SCHEMA = {
  "name": "css_skill_classifier",
  "schema": {
    "type": "object",
    "additionalProperties": False,
    "properties": {
      "is_skill": {"type": "boolean"},
      "capability_name": {"type": "string"},
      "skill_name": {"type": "string"},
      "service_interface": {"type": "string"},
      "reason_short": {"type": "string"},
      "evidence": {
        "type": "array",
        "items": {"type": "string"},
        "minItems": 1,
        "maxItems": 6
      },
      "inputs_guess": {"type": "array", "items": {"type": "string"}},
      "outputs_guess": {"type": "array", "items": {"type": "string"}}
    },
    "required": ["is_skill", "capability_name", "skill_name", "service_interface", "reason_short", "evidence", "inputs_guess", "outputs_guess"]
  }
}



In [ ]:
import json

EXPECTED_KEYS = [
    "is_skill",
    "capability_name_realworld",
    "skill_name",
    "service_interface",
    "real_world_effect_guess",
    "actuators_guess",
    "sensors_guess",
    "reason_short",
    "evidence",
    "inputs_guess",
    "outputs_guess",
]

def classify_pou_as_css_skill(pou_row: dict, model: str = "llama-3.1-8b-instant"):
    code = pou_row["code"] or ""

    prompt = f"""
{CSS_DEFINITION}

Gib ausschließlich EIN JSON-Objekt zurück (kein Markdown, kein Text außenrum).
Das JSON MUSS genau diese Keys enthalten:
{EXPECTED_KEYS}

Regeln:
- capability_name_realworld: 3 bis 8 Wörter, deutsch, beschreibt die physische Wirkung (z.B. "Vertikalachse encodergeführt bewegen").
  NICHT erlaubt: "abstrakt", "n/a", "unknown", "Skill", "Capability".
- real_world_effect_guess: 1 Satz, konkret: was wird bewegt/geschaltet/gemessen? Nutze nur Hinweise aus dem Code.
- actuators_guess: Liste von Aktor-Signalen/Outputs aus dem Code (z.B. DO_...).
- sensors_guess: Liste von Sensor/Encoder/DI Signalen aus dem Code (z.B. TouchSensor..., Encoder...).
- evidence: 2 bis 6 kurze Code-Zeilen wörtlich, die deine Aussage stützen.

POU-Name: {pou_row["name"]}
Sprache: {pou_row["lang"]}

Code:
{code}
""".strip()

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Return valid JSON only."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
        max_completion_tokens=600,
        response_format={"type": "json_object"},  # <-- wichtig
    )

    data = json.loads(resp.choices[0].message.content)

    # Minimal-Validation (damit du merkst, wenn das Modell "ausbricht")
    missing = [k for k in EXPECTED_KEYS if k not in data]
    if missing:
        raise ValueError(f"Missing keys in JSON: {missing}")

    return data


In [19]:
results = []
for i, r in enumerate(rows, start=1):
    try:
        out = classify_pou_as_css_skill(r)
        out["_pou_iri"] = r["pou_iri"]
        out["_pou_lang"] = r["lang"]
        results.append(out)
        print(f"[{i}/{len(rows)}] {r['name']} -> is_skill={out['is_skill']}")
        time.sleep(0.2)  # optional: etwas drosseln
    except Exception as e:
        print(f"[{i}/{len(rows)}] ERROR bei {r['name']}: {e}")

with open("css_skill_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Fertig. Ergebnisse:", len(results))



[1/22] FB_MyOpcUaMethod -> is_skill=True
[2/22] JobMethode_Schablone -> is_skill=True
[3/22] MBS_CB_AS_HorizontalMove -> is_skill=True
[4/22] MBS_DmPD_AS_HorizontalMove -> is_skill=True
[5/22] MBS_MR01_AS_HardeningProcess -> is_skill=True
[6/22] MBS_MR02_AS_CuttingProcess -> is_skill=True
[7/22] MBS_VSG_AS_VerticalMove -> is_skill=True
[8/22] SST_CS_AS_ColorDetection -> is_skill=True
[9/22] VSG_AS_CompressorControl -> is_skill=True
[10/22] HRL_CB_AS_HorizontalMoveSensors -> is_skill=True
[11/22] HRL_RGB_AS_HorizontalMoveEncoders -> is_skill=True
[12/22] HRL_RGB_AS_HorizontalMoveSensors -> is_skill=True
[13/22] HRL_RGB_AS_VerticalMoveEncoders -> is_skill=True
[14/22] MBS_MR01_AS_HorizontalMoveSensors -> is_skill=True
[15/22] MBS_VSG_AS_HorizontalMove -> is_skill=True
[16/22] MBS_VSG_AS_SuctionProcess -> is_skill=True
[17/22] VSG_AS_HorizontalMoveEncoder -> is_skill=True
[18/22] VSG_AS_RotationMoveEncoder -> is_skill=True
[19/22] VSG_AS_VerticalMoveEncoder -> is_skill=True
[20/22] SST_PD